# Utility

In [ ]:
!pip install -q gdown
!pip install quanto

In [ ]:
import gdown
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
import warnings
import string
import torch
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_scheduler
from tqdm import tqdm
from sklearn.metrics import classification_report
import torch.nn.functional as F
import matplotlib.pyplot as plt
import gc
import os
import torch.quantization as quant

In [ ]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    size_mb = os.path.getsize("temp.p") / 1e6
    os.remove("temp.p")
    return size_mb

# Pre-Trained Model Loading

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load the saved model state
model.load_state_dict(torch.load('model.pt'))
model.eval()

<ipython-input-7-72d2fab971ac>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pt'))


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# Print fine-tuned model size
model_size = print_size_of_model(model)
print(f"Quantized model size: {model_size:.2f} MB")

Quantized model size: 438.00 MB


# Post Training Quantization (PTQ)

In [ ]:
# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [ ]:
# Print quantized model size
quantized_size = print_size_of_model(quantized_model)
print(f"Quantized model size: {quantized_size:.2f} MB")

Quantized model size: 181.48 MB


In [ ]:
torch.save(model.state_dict(), 'quantized_model.pt')

In [ ]:
from google.colab import files
files.download('quantized_model.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Real Life Example

Load the Mode

In [ ]:
headline = 'For Russia, Nuclear Weapons Are the Ultimate Bargaining Chip'

In [ ]:
body = """On the 1,000th day of the war in Ukraine, President Volodymyr Zelensky took advantage of Washington’s new willingness to allow long-range missiles to be shot deep into Russia. Until this weekend, President Biden had declined to allow such strikes using American weapons, out of fear they could prompt World War III.

On the same day, Russia formally announced a new nuclear doctrine that it had signaled two months ago, declaring for the first time that it would use nuclear weapons not only in response to an attack that threatened its survival, but also in response to any attack that posed a “critical threat” to its sovereignty and territorial integrity — a situation very similar to what was playing out in the Kursk region, as American-made ballistic missiles struck Russian weapons arsenals.

And there was another wrinkle to Russia’s guidelines for nuclear use: For the first time, it declared the right to use nuclear weapons against a state that only possesses conventional arms — if it is backed by a nuclear power. Ukraine, backed by the United States, Britain and France — three of the five original nuclear-armed states — seems to be the country Russia’s president, Vladimir V. Putin, had in mind.

Yet it was telling that the reaction in Washington on Tuesday was just short of a yawn. Officials dismissed the doctrine as the nothingburger of nuclear threats. Instead, the city was rife with speculation over who would prevail as Treasury secretary, or whether Matt Gaetz, a former congressman surrounded by sex-and-drug allegations though never charged, could survive the confirmation process to become attorney general.

The Ukraine war has changed many things: It has ended hundreds of thousands of lives and shattered millions, it has shaken Europe, and it has deepened the enmity between Russia and the United States. But it has also inured Washington and the world to the renewed use of nuclear weapons as the ultimate bargaining chip. The idea that one of the nine countries now in possession of nuclear weapons — with Iran on the threshold of becoming the tenth — might press the button is more likely to evoke shrugs than a convening of the United Nations Security Council.

“This is a signaling exercise, trying to scare audiences in Europe — and to a lesser extent, the United States — into falling off support for Ukraine,” said Matthew Bunn, a Harvard professor who has tracked nuclear risks for decades. “The actual short-term probability of Russian nuclear use hasn’t increased. The long-term probability of nuclear war has probably increased slightly — because U.S. willingness to support strikes deep into Russia is reinforcing Putin’s hatred and fear of the West, and will likely provoke Russian responses that will increase Western fear and hatred of Russia.”"""


In [ ]:
data = [f'{headline} {body}']
encodings = tokenizer(data, truncation=True, padding=True, max_length=128, return_tensors="pt")

In [ ]:
# Move model to the correct device
quantized_model.to('cpu')
quantized_model.eval()

# Move input tensors (encodings) to the same device
encodings = {key: val.to('cpu') for key, val in encodings.items()}

In [ ]:
predictions = []

with torch.no_grad():
    outputs = quantized_model(**encodings)  # No need to call .to(device) on outputs
    logits = outputs.logits

    # Get predictions
    predicted_class = torch.argmax(logits, dim=-1)
    predictions.extend(predicted_class.cpu().numpy())  # Move predictions to CPU for numpy compatibility

print("Predictions:", predictions)

Predictions: [2]


In [ ]:
probabilities = F.softmax(logits, dim=1)
percentages = probabilities * 100
print(percentages)

tensor([[28.2651, 31.6096, 40.1253]])


In [ ]:
predictions

[2]